<a href="https://colab.research.google.com/github/Nuri-Tas/NLP/blob/main/Text%20Classification/Cipher_Decryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cipher Decryption

We'll build a decrypter following a genetic programming approach. We'll start with building simple encoder/decoders that will map each character to another, and we will evolve offspring from random mappings and only make the mappings with the highest likelihood to survive throughout generations.

In [1]:
import string
from string import ascii_lowercase
import random
import numpy as np

In [2]:
# we are also goint to add space character to our alphabet
alphabet = ascii_lowercase + " "

shuffled_alphabet = "".join(np.random.choice(list(alphabet), size=len(alphabet), replace=False))
print(shuffled_alphabet)

czpfhuaovtn eixkmjwldsbrqgy


In [3]:
cypher_dict = dict(zip(alphabet, shuffled_alphabet))
decode_dict = {v: k for k, v in cypher_dict.items()}

def encode(text):
  chars = list(text.lower())
  output = ""
  for char in chars:
    output += cypher_dict[char]
  return output

def decode(text):
  chars = list(text.lower())
  output = ""
  for char in chars:
    output += decode_dict[char]
  return output

In [4]:
encode("top secret")

'lxkywhpjhl'

In [5]:
decode("wsul dyodw")

'svftlu hus'

We are going to use moby_dict text to develop the markov matrix.

In [6]:
!wget -nc https://lazyprogrammer.me/course_files/moby_dick.txt

--2023-04-16 13:59:20--  https://lazyprogrammer.me/course_files/moby_dick.txt
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘moby_dick.txt’

moby_dick.txt           [ <=>                ]   1.17M  --.-KB/s    in 0.1s    

2023-04-16 13:59:20 (9.13 MB/s) - ‘moby_dick.txt’ saved [1227649]



In [7]:
with open("/content/moby_dick.txt") as dick_file:
   moby = dick_file.read()

We will remove every punctuation except periods to be able to find unigram -starting words in our case- distribution.

In [8]:
def preprocess(text, period_include=True):
  if period_include:
    return text.translate(str.maketrans("", "", string.punctuation)).lower()
  else:
    return text.translate(str.maketrans("", "", string.punctuation.replace(".", "") )).lower()

In [9]:
moby = preprocess(moby, False)

Our goal is to find the secret decypher. We're going to encode the following text with secret decypher and try to find its original form off the decoded text.


In [10]:
TEXT_TO_ENCODE = """I then lounged down the street and found,
as I expected, that there was a mews in a lane which runs down
by one wall of the garden. I lent the ostlers a hand in rubbing
down their horses, and received in exchange twopence, a glass of
half-and-half, two fills of shag tobacco, and as much information
as I could desire about Miss Adler, to say nothing of half a dozen
other people in the neighbourhood in whom I was not in the least
interested, but whose biographies I was compelled to listen to."""

In [11]:
alphabet = list(string.ascii_lowercase)

def get_random_map():
  shuffled_alphabet = np.random.choice(alphabet, size=len(alphabet), replace=False)
  decypher = {k: v for k, v in zip(alphabet, shuffled_alphabet)}
  return decypher

secret_decypher = get_random_map()

def encode(text, mapping):
    encoded = text.lower().translate(str.maketrans("".join(mapping.keys()), "".join(mapping.values())))
    return encoded

def decode(text, mapping):
    decoded = text.lower().translate(str.maketrans("".join(mapping.values()), "".join(mapping.keys())))
    return decoded

encoded_text = encode(TEXT_TO_ENCODE, secret_decypher)

# the encoded text we will be working on 
print(encoded_text)

n eqdc ibscodj jbkc eqd wefdde ucj abscj,
uw n dgxdledj, eque eqdfd kuw u tdkw nc u iucd kqnlq fscw jbkc
zp bcd kuii ba eqd oufjdc. n idce eqd bweidfw u qucj nc fszznco
jbkc eqdnf qbfwdw, ucj fdldnvdj nc dglqucod ekbxdcld, u oiuww ba
quia-ucj-quia, ekb aniiw ba wquo ebzullb, ucj uw tslq ncabftuenbc
uw n lbsij jdwnfd uzbse tnww ujidf, eb wup cbeqnco ba quia u jbydc
beqdf xdbxid nc eqd cdnoqzbsfqbbj nc kqbt n kuw cbe nc eqd iduwe
ncedfdwedj, zse kqbwd znbofuxqndw n kuw lbtxdiidj eb inwedc eb.


In [12]:
random_map = get_random_map()

random_decoded = decode(encoded_text, random_map)
print(random_decoded)

s yorh umkhlrv vmdh yor qyarry thv emkhv,
tq s rnpriyrv, yoty yorar dtq t xrdq sh t uthr dosio akhq vmdh
zj mhr dtuu me yor ltavrh. s urhy yor mqyuraq t othv sh akzzshl
vmdh yorsa omaqrq, thv arirsbrv sh rniothlr ydmprhir, t lutqq me
otue-thv-otue, ydm esuuq me qotl ymztiim, thv tq xkio shemaxtysmh
tq s imkuv vrqsar tzmky xsqq tvura, ym qtj hmyoshl me otue t vmfrh
myora prmpur sh yor hrslozmkaommv sh domx s dtq hmy sh yor urtqy
shyrarqyrv, zky domqr zsmlatposrq s dtq imxpruurv ym usqyrh ym.


Let's now create bigrams based on characters for the entire corpus. We'll initialize a 26x26 Markov matrix and represent the characters in their respective order. Similarly, we will store the probability of starting character in the pi vector.

In [13]:
chars_idx = {char: alphabet.index(char) for char in alphabet}

V = len(alphabet)
pi = np.ones(V)
A = np.ones((V, V))

chars = [chars_idx.get(item, -1) for item in list(moby)]
for i in range(len(chars)-1):
  first, second = chars[i], chars[i+1]
  if (first != -1) and (second != -1):
    A[first, second] += 1
  elif (first == -1) and (second != -1):
    pi[second] += 1

# normalize matrixes and dampen them
pi /= pi.sum()
A /=  A.sum()

logpi = np.log10(pi)
logA = np.log10(A)

We will calculate the log likelihood of each decoded text we obtained.

In [14]:
def log_likelihood(text):
  prob = 0
  text_chars = [chars_idx.get(item, -1) for item in list(text)]
  for i in range(len(text_chars)-1):
    first, second = text_chars[i], text_chars[i+1]
    if (first != -1) and (second != -1):
      prob += logA[first, second]
    if (first == -1) and (second != -1):
      prob += logpi[second]
  
  return prob

In [15]:
log_likelihood("hello")

-8.114388885842335

In [16]:
log_likelihood("adsfd")

-14.631202932957311

As expected, the log likelihood of the first random decoded text will result in a great negative number.

In [17]:
log_likelihood(random_decoded)

-1224.6547874294424

We are going to create 3 children per parent by swapping only two characters.

In [18]:
dna_pool = []
for _ in range(20):
  parent = np.random.choice(alphabet, size=len(alphabet), replace=False)
  dna_pool.append(parent)

def children(dna_pool, no_of_children):
  children_pool = []
  for dna in dna_pool:
    for _ in range(no_of_children):
      dna_copy = dna.copy()
      i, j = np.random.randint(26), np.random.randint(26)
      dna_copy[i], dna_copy[j] = dna_copy[j], dna_copy[i] 
      children_pool.append(dna_copy)
  
  return dna_pool + children_pool 

At each iteration, we will only save the first 5 mappings with the highest score, and evolve offsprings for 1000 generations. 

In [19]:
best_dna = None
best_map = None
best_likelihood = float("-inf")

for i in range(1000):
  if i > 0:
    dna_pool = children(dna_pool, 3)

  scores = {}
  for dna in dna_pool:
    new_map = {k: v for k, v in zip(alphabet, dna)}
    new_likelihood = log_likelihood(decode(encoded_text, new_map))
    scores["".join(dna)] = new_likelihood

    if new_likelihood > best_likelihood:
      best_dna = dna  
      best_likelihood = new_likelihood
      best_map = new_map

    # only let the first five dna with the highest likelihood to survive
    dna_sorted = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    dna_pool = [list(k) for k, v in dna_sorted[:5]]

  if (i % 100) == 0:
    print("Best score so far:", best_likelihood)

Best score so far: -1223.6839746156252
Best score so far: -930.3637479029846
Best score so far: -900.7511150978613
Best score so far: -898.8440007532588
Best score so far: -898.8440007532588
Best score so far: -898.8440007532588
Best score so far: -898.8440007532588
Best score so far: -898.8440007532588
Best score so far: -898.8440007532588
Best score so far: -898.8440007532588


## Final Result

Our final decypeher yielded in the following decoded text, which although contains little mistakes, achieved an impressive result. 

In [20]:
# decoded text from the map we built
best_text = decode(encoded_text, best_map)
best_text

's atri olmicrn nlwi atr eaurra hin dlmin,\nhe s rkfrparn, atha atrur whe h grwe si h ohir wtspt umie nlwi\nby lir whoo ld atr chunri. s oria atr leaorue h thin si umbbsic\nnlwi atrsu tluere, hin urprsqrn si rkpthicr awlfripr, h cohee ld\nthod-hin-thod, awl dsooe ld ethc albhppl, hin he gmpt sidlughasli\nhe s plmon nresur hblma gsee hnoru, al ehy ilatsic ld thod h nlvri\nlatru frlfor si atr irsctblmutlln si wtlg s whe ila si atr orhea\nsiarurearn, bma wtler bslcuhftsre s whe plgfroorn al oseari al.'

In [21]:
# actual encoded text
actual_text = decode(encoded_text, secret_decypher)
actual_text

'i then lounged down the street and found,\nas i expected, that there was a mews in a lane which runs down\nby one wall of the garden. i lent the ostlers a hand in rubbing\ndown their horses, and received in exchange twopence, a glass of\nhalf-and-half, two fills of shag tobacco, and as much information\nas i could desire about miss adler, to say nothing of half a dozen\nother people in the neighbourhood in whom i was not in the least\ninterested, but whose biographies i was compelled to listen to.'

We can also realize the small difference on likelihood values as well. Evidently, the likelihood of the best text is higher than the actual text.

In [22]:
print("Likelihood of the actual text:",  log_likelihood(actual_text), "Likelihood of the best text:",  log_likelihood(best_text), sep="\n")

Likelihood of the actual text:
-769.4198483288895
Likelihood of the best text:
-898.8440007532588
